<a href="https://colab.research.google.com/github/woongchi98/jupyterML-DL/blob/main/FinalReport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
cd '/content/drive/MyDrive/Colab Notebooks/ai_contest'

[Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/ai_contest'
/content


In [14]:
!unzip rawdata.zip

Archive:  rawdata.zip
   creating: Test/
  inflating: Test/1000.TXT           
  inflating: Test/1001.TXT           
  inflating: Test/1002.TXT           
  inflating: Test/1003.TXT           
  inflating: Test/1004.TXT           
  inflating: Test/1005.TXT           
  inflating: Test/1006.TXT           
  inflating: Test/1007.TXT           
  inflating: Test/1008.TXT           
  inflating: Test/1009.TXT           
  inflating: Test/1010.TXT           
  inflating: Test/1011.TXT           
  inflating: Test/1012.TXT           
  inflating: Test/1013.TXT           
  inflating: Test/1014.TXT           
  inflating: Test/1015.TXT           
  inflating: Test/1016.TXT           
  inflating: Test/1017.TXT           
  inflating: Test/1018.TXT           
  inflating: Test/1019.TXT           
  inflating: Test/1020.TXT           
  inflating: Test/1021.TXT           
  inflating: Test/1022.TXT           
  inflating: Test/1023.TXT           
  inflating: Test/1024.TXT           
  inflati

In [2]:
import numpy as np
import os

def get_max_len(folderpath_train, folderpath_test):
    nMaxLen = 0
    for filename in os.listdir(folderpath_train):
        filepath = f'{folderpath_train}/{filename}'
        d, tmp = load_aSig(filepath)
        if d.shape[0] > nMaxLen:
            nMaxLen = d.shape[0]

    for filename in os.listdir(folderpath_test):
        filepath = f'{folderpath_test}/{filename}'
        d, tmp = load_aSig(filepath)
        if d.shape[0] > nMaxLen:
            nMaxLen = d.shape[0]

    return nMaxLen

def load_aSig(filepath):
    f = open(filepath,'rt')
    nPoints = int(f.readline())
    d = np.zeros((nPoints,2))
    for i in range(nPoints):
        line = f.readline()
        toks = line.split(' ')
        d[i,:] = [int(toks[0]), int(toks[1])]
    f.close()
    return d, nPoints

def load_data(folderpath,nMaxLen, sig_ids):
    filelist = os.listdir(folderpath)
    nFiles = len(filelist)
    d = np.zeros((nFiles,nMaxLen,2))
    len_sign = np.zeros(nFiles)

    for i in sig_ids:
        filepath = f'{folderpath}/{i}.TXT'
        d_tmp, len_sign[i-sig_ids[0]] = load_aSig(filepath)
        d[i-sig_ids[0],:d_tmp.shape[0],:] = d_tmp
    return d, len_sign

In [3]:
import pandas as pd
import numpy as np

from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras import Sequential

from tensorflow.keras.optimizers import Adam

In [92]:
#데이터 읽어오기
folderpath= '/content/drive/MyDrive/Colab Notebooks/ai_contest'

#각 데이터의 길이를 체크하고 최대길이 확인
n_max_len = get_max_len( f'{folderpath}/Train', f'{folderpath}/Test') 

#학습용 데이터 load
x_train, x_train_len = load_data(f'{folderpath}/Train', n_max_len, range(800))  
#테스트용 데이터 load 
x_test, x_test_len = load_data(f'{folderpath}/Test', n_max_len, range(800,1600)) 

#학습용 데이터 변환
y_train_pd = pd.read_csv(f'{folderpath}/train_info.csv')
y_train = np.array(y_train_pd['bGenuine'])
y_train_cat = keras.utils.to_categorical(y_train)

In [94]:
#Robust Scaling
for i in range(x_train.shape[0]): 
  len_tmp = int(x_train_len[i])
  h_min = np.quantile(x_train[i,:len_tmp,0],0.25)
  h_max = np.quantile(x_train[i,:len_tmp,0],0.75)
  v_min = np.quantile(x_train[i,:len_tmp,1],0.25)
  v_max = np.quantile(x_train[i,:len_tmp,1],0.75)

  w_h_ratio = (v_max - v_min)/ (h_max - h_min) #너비-높이 비율

  x_train[i,:len_tmp,0] = (x_train[i,:len_tmp,0] - h_min) / (h_max - h_min) 
  x_train[i,:len_tmp,1] = (x_train[i,:len_tmp,1] - v_min) / (v_max - v_min) * w_h_ratio 

for i in range(x_test.shape[0]):
  len_tmp2 = int(x_test_len[i])
  h_min = np.quantile(x_test[i,:len_tmp,0],0.25)
  h_max = np.quantile(x_test[i,:len_tmp,0],0.75)
  v_min = np.quantile(x_test[i,:len_tmp,1],0.25)
  v_max = np.quantile(x_test[i,:len_tmp,1],0.75)

  w_h_ratio = (v_max - v_min)/ (h_max - h_min) #너비-높이 비율

  x_test[i,:len_tmp,0] = (x_test[i,:len_tmp,0] - h_min) / (h_max - h_min) 
  x_test[i,:len_tmp,1] = (x_test[i,:len_tmp,1] - v_min) / (v_max - v_min) * w_h_ratio 

In [80]:
# #정규화
# for i in range(x_train.shape[0]): 
#   len_tmp = int(x_train_len[i])
#   h_min = np.min(x_train[i,:len_tmp,0])
#   h_max = np.max(x_train[i,:len_tmp,0])
#   v_min = np.min(x_train[i,:len_tmp,1])
#   v_max = np.max(x_train[i,:len_tmp,1])

#   w_h_ratio = (v_max - v_min)/ (h_max - h_min) #너비-높이 비율

#   x_train[i,:len_tmp,0] = (x_train[i,:len_tmp,0] - h_min) / (h_max - h_min) 
#   x_train[i,:len_tmp,1] = (x_train[i,:len_tmp,1] - v_min) / (v_max - v_min) * w_h_ratio 
  

# for i in range(x_test.shape[0]):
#   len_tmp2 = int(x_test_len[i])
#   h_min = np.min(x_test[i,:len_tmp,0])
#   h_max = np.max(x_test[i,:len_tmp,0])
#   v_min = np.min(x_test[i,:len_tmp,1])
#   v_max = np.max(x_test[i,:len_tmp,1])

#   w_h_ratio = (v_max - v_min)/ (h_max - h_min) #너비-높이 비율

#   x_test[i,:len_tmp,0] = (x_test[i,:len_tmp,0] - h_min) / (h_max - h_min) 
#   x_test[i,:len_tmp,1] = (x_test[i,:len_tmp,1] - v_min) / (v_max - v_min) * w_h_ratio 

In [69]:
# ii = 3
# len_tmp_ex = int(x_train_len[ii])
# mi = np.quantile(x_train[ii ,:len_tmp_ex, 0], 0.25)
# ma = np.quantile(x_train[ii ,:len_tmp_ex, 0], 0.75)
# ma-mi

2043.25

In [95]:
filelist = os.listdir(folderpath + '/Train')
n_train = len(filelist)

In [96]:
y_pd = pd.read_csv(f'{folderpath}/train_info.csv')
y_all = np.array(y_pd['bGenuine'])
y_train = y_all[:n_train]
y_train_cat = keras.utils.to_categorical(y_train)
y_test = y_all[n_train:]

In [98]:
from tensorflow.keras.layers import *
from tensorflow.keras import Sequential

#모델 구조 지정
model = Sequential()
model.add(Conv1D(8, kernel_size=3, padding='same', activation='relu'))
model.add(Conv1D(8, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=3, strides=2))

model.add(Conv1D(16, kernel_size=3, padding='same', activation='relu'))
model.add(Conv1D(16, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=3, strides=2, input_dim=2))

model.add(Conv1D(32, kernel_size=3, padding='same', activation='relu'))
model.add(Conv1D(32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=3, strides=2, input_dim=2))

model.add(Bidirectional(LSTM(128)))

model.add(Dropout(0.2))
model.add(Dense(units=2, activation='softmax'))

In [ ]:
#모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

#학습
hist = model.fit(x=x_train, y=y_train_cat,	epochs=100, validation_split=0.2, batch_size=16)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/100
40/40 [==============================] - 13s 216ms/step - loss: 0.6950 - accuracy: 0.5078 - val_loss: 0.6926 - val_accuracy: 0.5063
Epoch 2/100
40/40 [==============================] - 7s 186ms/step - loss: 0.6902 - accuracy: 0.5281 - val_loss: 0.7014 - val_accuracy: 0.4875
Epoch 3/100
40/40 [==============================] - 7s 183ms/step - loss: 0.6660 - accuracy: 0.5813 - val_loss: 0.6506 - val_accuracy: 0.5813
Epoch 4/100
40/40 [==============================] - 7s 178ms/step - loss: 0.6457 - accuracy: 0.6219 - val_loss: 0.6585 - val_accuracy: 0.5625
Epoch 5/100
40/40 [==============================] - 7s 180ms/step - loss: 0.6352 - accuracy: 0.6297 - val_loss: 0.6763 - val_accuracy: 0.5750
Epoch 6/100
40/40 [==============================] - 7s 177ms/step - loss: 0.6211 - accuracy: 0.6484 - val_loss: 0.6506 - val_accuracy: 0.5875
Epoch 7/100
40/40 [==============================] - 7s 181ms/step - loss: 0.6351 - accuracy: 0.6391 - val_loss: 0.6317 - val_accuracy: 0.637

In [90]:
#테스트
o = model.predict(x_test)
o = np.argmax(o,1)

In [91]:
#제출파일
answer = {'id' : list(range(800,1600)), 'predicted' : o}
submission = pd.DataFrame(data=answer)
submission.to_csv('output5.csv', index=False)

In [ ]:
z